In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_ui.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fprompt_optimizer%2Fvertex_ai_prompt_optimizer_ui.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_ui.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_ui.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

# Overview
Welcome to Vertex AI Prompt Optimizer (VAPO)! This Notebook showcases VAPO, a tool that iteratively optimizes prompts to suit a target model (e.g., `gemini-1.5-pro`) using target-specific metric(s).

Key Use Cases:

* Prompt Optimization: Enhance the quality of an initial prompt by refining its structure and content to match the target model's optimal input characteristics.

* Prompt Translation: Adapt prompts optimized for one model to work effectively with a different target model.

# Step 0: Install packages and libraries

In [ ]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py
import vapo_lib

# Step 1: Configure your prompt template
Prompts consist of two key parts:
* System Instruction (SI) Template: A fixed instruction shared across all queries for a given task.
* Task/Context Template: A dynamic part that changes based on the task.

APD enables the translation and optimization of the System Instruction Template, while the Task/Context Template remains essential for evaluating different SI templates.

In [ ]:
SYSTEM_INSTRUCTION = "Answer the following question. Let's think step by step.\n"  # @param {type:"string"}
PROMPT_TEMPLATE = "Question: {question}\n\nAnswer:{target}"  # @param {type:"string"}

# Step 2: Input your data
To optimize the model, provide a CSV or JSONL file containing labeled validation samples
* Focus on examples that specifically demonstrate the issues you want to address.
* Recommendation: Use 50-100 distinct samples for reliable results. However, the tool can still be effective with as few as 5 samples.

For prompt translation:
* Consider using the source model to label examples that the target model struggles with, helping to identify areas for improvement.


In [ ]:
# @markdown **Project setup**: <br/>
PROJECT_ID = "[YOUR_PROJECT]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
OUTPUT_PATH = "[OUTPUT_PATH]"  # @param {type:"string"}
# @markdown * GCS path of your bucket, e.g., gs://prompt_translation_demo, used to store all artifacts.
INPUT_DATA_PATH = "[INPUT_DATA_PATH]"  # @param {type:"string"}
# @markdown * Specify a GCS path for the input data, e.g., gs://prompt_translation_demo/input_data.jsonl.

# Step 3: Configure optimization settings
The optimization configs are defaulted to the values that are most commonly used and which we recommend using initially.

In [ ]:
SOURCE_MODEL = ""  # @param ["", "gemini-1.0-pro-001", "gemini-1.0-pro-002", "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-ultra-001", "text-bison@001", "text-bison@002", "text-bison32k@002", "text-unicorn@001"]
# @markdown * If set, it will be used to generate ground truth responses for the input examples. This is useful to migrate the prompt from a source model.
TARGET_MODEL = "gemini-1.5-flash-001"  # @param ["gemini-1.0-pro-001", "gemini-1.0-pro-002", "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-ultra-001"]
OPTIMIZATION_MODE = "instruction_and_demo"  # @param ["instruction", "demonstration", "instruction_and_demo"]
EVAL_METRIC = "question_answering_correctness"  # @param ["bleu", "coherence", "exact_match", "fluency", "groundedness", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}

# Step 4: Configure advanced optimization settings [Optional]

In [ ]:
# @markdown **Instruction Optimization Configs**: <br/>
NUM_INST_OPTIMIZATION_STEPS = 10  # @param {type:"integer"}
NUM_TEMPLATES_PER_STEP = 2  # @param {type:"integer"}
# @markdown * Number of prompt templates generated and evaluated at each optimization step.

# @markdown **Demonstration Optimization Configs**: <br/>
NUM_DEMO_OPTIMIZATION_STEPS = 10  # @param {type:"integer"}
NUM_DEMO_PER_PROMPT = 3  # @param {type:"integer"}
# @markdown * Number of the demonstrations to include in each prompt.

# @markdown **Model Configs**: <br/>
TARGET_MODEL_QPS = 3.0  # @param {type:"number"}
SOURCE_MODEL_QPS = 3.0  # @param {type:"number"}
EVAL_QPS = 3.0  # @param {type:"number"}
# @markdown * The QPS for calling the eval model, which is currently gemini-1.5-pro-001.

# @markdown **Multi-metric Configs**: <br/>
# @markdown Use this section only if you need more than one metric for optimization. This will override the metric you picked above.
EVAL_METRIC_1 = "NA"  # @param ["NA", "bleu", "coherence", "exact_match", "fluency", "groundedness", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_1_WEIGHT = 0.0  # @param {type:"number"}
EVAL_METRIC_2 = "NA"  # @param ["NA", "bleu", "coherence", "exact_match", "fluency", "groundedness", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_2_WEIGHT = 0.0  # @param {type:"number"}
EVAL_METRIC_3 = "NA"  # @param ["NA", "bleu", "coherence", "exact_match", "fluency", "groundedness", "text_quality", "verbosity", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_quality", "summarization_quality", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match", "tool_call_valid"] {type:"string"}
EVAL_METRIC_3_WEIGHT = 0.0  # @param {type:"number"}
METRIC_AGGREGATION_TYPE = "weighted_sum"  # @param ["weighted_sum", "weighted_average"]

# @markdown **Misc Configs**: <br/>
PLACEHOLDER_TO_VALUE = "{}"  # @param
# @markdown * This variable is used for long prompt optimization to not optimize parts of prompt identified by placeholders. It provides a mapping from the placeholder variables to their content. See link for details.
RESPONSE_MIME_TYPE = "application/json"  # @param ["text/plain", "application/json"]
# @markdown * This variable determines the format of the output for the target model. See link for details.
TARGET_LANGUAGE = "English"  # @param ["English", "French", "German", "Hebrew", "Hindi", "Japanese", "Korean", "Portuguese", "Simplified Chinese", "Spanish", "Traditional Chinese"]
# @markdown * The language of the system instruction.

# Step 5: Run Prompt Optimizer

In [ ]:
import datetime
import json
import os
import time

from google.colab import auth

timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
display_name = f"pt_{timestamp}"

in_colab_enterprise = "GOOGLE_CLOUD_PROJECT" in os.environ
if not in_colab_enterprise:
    auth.authenticate_user()

label_enforced = vapo_lib.is_run_target_required(
    [
        EVAL_METRIC,
        EVAL_METRIC_1,
        EVAL_METRIC_2,
        EVAL_METRIC_3,
    ],
    SOURCE_MODEL,
)
input_data_path = f"{INPUT_DATA_PATH}"
vapo_lib.validate_prompt_and_data(
    "\n".join([SYSTEM_INSTRUCTION, PROMPT_TEMPLATE]),
    input_data_path,
    PLACEHOLDER_TO_VALUE,
    label_enforced,
)

output_path = f"{OUTPUT_PATH}/{display_name}"

params = {
    "project": PROJECT_ID,
    "num_steps": NUM_INST_OPTIMIZATION_STEPS,
    "system_instruction": SYSTEM_INSTRUCTION,
    "prompt_template": PROMPT_TEMPLATE,
    "target_model": TARGET_MODEL,
    "target_model_qps": TARGET_MODEL_QPS,
    "target_model_location": LOCATION,
    "source_model": SOURCE_MODEL,
    "source_model_qps": SOURCE_MODEL_QPS,
    "source_model_location": LOCATION,
    "eval_qps": EVAL_QPS,
    "eval_model_location": LOCATION,
    "optimization_mode": OPTIMIZATION_MODE,
    "num_demo_set_candidates": NUM_DEMO_OPTIMIZATION_STEPS,
    "demo_set_size": NUM_DEMO_PER_PROMPT,
    "aggregation_type": METRIC_AGGREGATION_TYPE,
    "data_limit": 50,
    "num_template_eval_per_step": NUM_TEMPLATES_PER_STEP,
    "input_data_path": input_data_path,
    "output_path": output_path,
    "response_mime_type": RESPONSE_MIME_TYPE,
    "language": TARGET_LANGUAGE,
    "placeholder_to_content": json.loads(PLACEHOLDER_TO_VALUE),
}

if EVAL_METRIC_1 == "NA":
    params["eval_metrics_types"] = [EVAL_METRIC]
    params["eval_metrics_weights"] = [1.0]
else:
    metrics = []
    weights = []
    for metric in [EVAL_METRIC_1, EVAL_METRIC_2, EVAL_METRIC_3]:
        if metric == "NA":
            break
        metrics.append(metric)
        weights.append(EVAL_METRIC_1_WEIGHT)
    params["eval_metrics_types"] = metrics
    params["eval_metrics_weights"] = weights

job = vapo_lib.run_apd(params, OUTPUT_PATH, display_name)
print(f"Job ID: {job.name}")

progress_form = vapo_lib.ProgressForm(params)
while progress_form.monitor_progress(job):
    time.sleep(5)

# Step 6: Inspect the Results
You can use the following cell to inspect all the predictions made by all the
generated templates during one or multiple VAPO runs.

In [ ]:
from IPython.display import HTML, display

RESULT_PATH = "[OUTPUT_PATH]"  # @param {type:"string"}
# @markdown * Specify a GCS path that contains artifacts of a single or multiple VAPO runs.

results_ui = vapo_lib.ResultsUI(RESULT_PATH)

results_df_html = """
<style>
  .scrollable {
    width: 100%;
    height: 80px;
    overflow-y: auto;
    overflow-x: hidden;  /* Hide horizontal scrollbar */
  }
  tr:nth-child(odd) {
    background: var(--colab-highlighted-surface-color);
  }
  tr:nth-child(even) {
    background-color: var(--colab-primary-surface-color);
  }
  th {
    background-color: var(--colab-highlighted-surface-color);
  }
</style>
"""

display(HTML(results_df_html))
display(results_ui.get_container())